## Загрузка библиотек и данных

In [239]:
import pandas as pd
import zipfile
import os
import time
import re
import nltk
import numpy as np

In [240]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize # плохо обработались слова
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer,  TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [241]:
!pip install pysrt
import pysrt

In [242]:
from google.colab import drive
from PIL import Image

In [243]:
!pip install pypdf
from pypdf import PdfReader

In [244]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Создание дф

In [271]:
# чтобы разгрузить код я буду путь к файлам сохранять в эту переменную
PATH = None

In [272]:
# создаем датафрейм
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/movies_labels.xlsx'
data = pd.read_excel(PATH)
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [273]:
# приведем названия столбцов к каноническому виду
data.columns = ['id', 'movie', 'level']
data.head()

,id,movie,level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [274]:
data.shape[0]

241

In [275]:
# читаем файл с субтитрами
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/Subtitles/10_Cloverfield_lane(2016).srt'
subs = pysrt.open(PATH)


In [276]:
# я посмотрела как будут выглядеть субтитры
subtitle_texts = [subtitle.text for subtitle in subs]
subtitle_texts[:10]

['<font color="#ffff80"><b>Fixed & Synced by bozxphd. Enjoy The Flick</b></font>',
 '(CLANGING)',
 '(DRAWER CLOSES)',
 '(INAUDIBLE)',
 '(CELL PHONE RINGING)',
 "BEN ON PHONE: <i>Michelle,<br/>please don't hang up.</i>",
 "<i>Just talk to me, okay?<br/>I can't believe you just left.</i>",
 '<i>Michelle.</i>',
 '<i>Come back.</i>',
 '<i>Please say something.</i>']

Для того, чтобы получить субтитры в удобноваримом формате, мы пройдемся циклом по всему содержимому папки Subtitles, которая содержит папки с названиями фильмов. Для этого используем модуль os, который позволяет работать с файлами и директориями.

In [277]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/'

In [278]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/'

def get_subs():
    subtitles = []
    movie = []

    # Получаем все папки и файлы в указанной папке
    film_list = os.listdir(PATH)

    for folder in film_list:
        for film in os.listdir(os.path.join(PATH, folder)):
            # достаем названия
            last_dot = film.rfind('.')
            movie.append(film[:last_dot])

            # субтитры
            # собираем название путь
            file_to_read = os.path.join(PATH, folder, film)  # Исправленный путь к файлу
            # расшифровываем
            try:
                encoded_film = pysrt.open(file_to_read, encoding='iso-8859-1')
            except:
                encoded_film = pysrt.open(file_to_read, encoding='UTF-8')

            # трансформируем в текст
            subs_text = [film.text for film in encoded_film]
            subtitles.append(subs_text)

    return pd.DataFrame({'subs': subtitles, 'movie': movie})


Обращаем внимание, что имеются строки в которых точка используется как разделитель, это важно, так как нам нужно сохранить названия фильмов без расширения .srt

---

Делить будем так:
- находим индекс где стоит **последняя** точка
- добавляем строку до этого индекса

In [279]:
subs_df = get_subs()
subs_df.head()

,subs,movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",The Walking Dead-S01E05-Wildfire.English
1,"[- ( people yelling )\n- ( radio chatter ), He...",The Walking Dead-S01E06-TS-19.English
2,"[( birds chirping ), - What?\n- Nothing., It's...",The Walking Dead-S01E04-Vatos.English
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",The Walking Dead-S01E03-Tell It To The Frogs.E...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",The Walking Dead-S01E02-Guts.English


In [280]:
subs_df.shape[0]

278

Теперь можно обработать названия фильмов, чтобы добавить субтитры к фильмам.
Обрабатывать буду следующими шагами:
- все слова к строчному регистру
- в качестве разделителя только знак нижнего подчеркивания "_"
- удалю все пробелы, знаки между словами, оставлю только цифры.

In [281]:
data.head()

,id,movie,level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [282]:
def transform_values(df):
    # заменяем пробелы на underscore
    df = df.applymap(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)

    # заменяем дефис на underscore
    df = df.applymap(lambda x: x.replace('-', '_') if isinstance(x, str) else x)

    # добавляем underscore между словом и цислом
    df = df.applymap(lambda x: '_'.join(part for part in x.split() if not part.isdigit()) if isinstance(x, str) else x)
    # приводим к строчному регистру
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    return df


In [283]:
# почистим названия фильмов в дата
data = transform_values(data)
data.head()

,id,movie,level
0,0,10_cloverfield_lane(2016),b1
1,1,10_things_i_hate_about_you(1999),b1
2,2,a_knights_tale(2001),b2
3,3,a_star_is_born(2018),b2
4,4,aladdin(1992),a2/a2+


In [284]:
subs_df = transform_values(subs_df)
subs_df.head()

,subs,movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english


In [285]:
subs_df['movie'] = transform_values(subs_df[['movie']])
subs_df['movie'].head()

0             the_walking_dead_s01e05_wildfire.english
1                the_walking_dead_s01e06_ts_19.english
2                the_walking_dead_s01e04_vatos.english
3    the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4                 the_walking_dead_s01e02_guts.english
Name: movie, dtype: object

In [286]:
subs_df.head()

,subs,movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english


Сделаю тип объединения outer так как могут удалиться дубликаты, а я хочу сохранить все наблюдения и отдельно обработать дубликаты чтобы избежать ошибок.

In [287]:
data = pd.merge (subs_df,data,
                 on = 'movie',
                 how = 'outer',
                 indicator = True)
data.head()

,subs,movie,id,level,_merge
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,116.0,a2,both
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,117.0,a2,both
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,115.0,a2,both
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,114.0,a2,both
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,113.0,a2,both


In [288]:
data['_merge'].value_counts()

both          233
left_only      49
right_only      8
Name: _merge, dtype: int64

видим что у нас остались 49 фильмов только из датафрейма subs_df и 8 только из data.
очевидно, что для 49 фильмов у нас нет разметки по уровню и для 8 нет субтитров.

In [289]:
data.loc[(data['_merge']=='left_only') | (data['_merge']=='right_only')]

,subs,movie,id,level,_merge
38,"[On the clothes we wear,, the people who make ...",the.true.cost.2015.bluray.720p.700mb.ganool.com,NaN,NaN,left_only
39,"[Yes., Ah, there you are. Come in., I'm practi...","crown,_the_s01e05___smoke_and_mirrors.en",NaN,NaN,left_only
40,"[[woman speaks French] Your Majesty., [planes ...","crown,_the_s01e07___scientia_potentia_est.en.sdh",NaN,NaN,left_only
41,[[music plays]\n<i>...and then I swing you aro...,"crown,_the_s01e10___gloriana.en.sdh",NaN,NaN,left_only
42,"[[child]<i> Gina! Gina! Ella!</i>, -Gina! Cari...","crown,_the_s01e03___windsor.en.sdh",NaN,NaN,left_only
43,"[- Fuel on.\n- Fuel on., Chocks are in positio...","crown,_the_s01e04___act_of_god.en",NaN,NaN,left_only
44,"[[fanfare plays], [man] <i>More than half a mi...","crown,_the_s01e06___gelignite.en.sdh",NaN,NaN,left_only
45,"[[coughing], [coughing continues], [spits], [e...","crown,_the_s01e01___wolferton_splash.en.sdh",NaN,NaN,left_only
46,"[<i>...and then I swing you around there.</i>,...","crown,_the_s01e10___gloriana.en",NaN,NaN,left_only
47,"[<i>âª If your sweetheart âª</i>, <i>âª Sen...","crown,_the_s01e09___assassins.en.sdh",NaN,NaN,left_only



reader = PdfReader('/content/drive/MyDrive/Masterskaya_2/Oxford_CEFR_level/American_Oxford_3000_by_CEFR_level.pdf')
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text

In [290]:
os.listdir('/content/drive/MyDrive/Masterskaya_2/Oxford_CEFR_level')

['The_Oxford_5000_by_CEFR_level.pdf',
 'The_Oxford_3000_by_CEFR_level.pdf',
 'American_Oxford_5000_by_CEFR_level.pdf',
 'American_Oxford_3000_by_CEFR_level.pdf']

нужно либо маску разделитель уровней прописать либо предварительно вручную разграничить по уровням, потому что пока непонятно как работать с этим форматом text

## Предобработка и анализ

In [291]:
data.dtypes

subs        object
movie       object
id         float64
level       object
_merge    category
dtype: object

## Пропуски

In [292]:
data.shape[0]

290

In [293]:
data = data.dropna()
data = data.reset_index (drop=True)
data.isna().sum()

subs      0
movie     0
id        0
level     0
_merge    0
dtype: int64

In [294]:
data.shape[0]

233

## Дупликаты

In [295]:
data['movie'].duplicated().sum()

4

In [296]:
duplicates = data[data['movie'].duplicated()]
duplicates

,subs,movie,id,level,_merge
110,[Advertise your product or brand here<br/>cont...,inside_out(2015),44.0,b1,both
134,"[EMT 1:\nAll right, people,\nlet's move! Let's...",powder(1995),68.0,b1,both
144,"[[CROWD CHEERING IN DISTANCE], LElGH ANNE: The...",the_blind_side(2009),84.0,b1,both
185,"[<font color=""#D900D9"">(AIRPLANE APPROACHES)</...",the_terminal(2004),99.0,"a2/a2+,_b1",both


Мы увидели только ошибочные дупликаты, удалять их не будем.

### Предобработка субтитров

In [297]:
data.head()

,subs,movie,id,level,_merge
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,116.0,a2,both
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,117.0,a2,both
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,115.0,a2,both
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,114.0,a2,both
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,113.0,a2,both


- перевести значения в признаке subs в формат str
- нужно создать дополнительный столбец, чтобы превратить субтитры в кортеж из слов
- удалить символы кроме пробела и '', так как по длине фраз так же можно сделать предположение о сложности текста
- привести слова к первоначальной форме (ед число, настоящее время)/ лемматизация
- привести все слова к нижнему регистру
- удалить стоп-слова


In [298]:
# заменим символы
data['subs'] = data['subs'].replace('\n', '\t', regex=True)
data['subs'][0][:5]

['- ( walkie-talkie squawks )\n- Rick: Morgan,',
 "I don't know if you're out there.",
 "I don't know\nif you can hear me.",
 "Maybe you're listening right now.",
 'I hope so.']

Видим, что остались символы \n без пробела, но на данном этапе не получилось их убрать не используя регулярные выражения.


---

data['sub'] содержит субтитры, которые представлены в формате список списков.

Каждый список (фильм) содержит список предложений, каждое предложение содержит список строк.

сначала нам нужно все эти строки (подсписки) соединить в одну строку, где каждый символ(разделен) пробелом.

Поэтому сначала мы получим слова из каждого подсписка в формате списка

  `[sub_words = []

    for sublist in data_column:

        for word in sublist.split():
        
            sub_words.append(word)]`.

Теперь нам нужно список собрать в строку, для этого объединим все элементы этого списка

```
' '.join([word for lists in word_list)
```

Теперь мы имеем субтитры записанные в строковом формате, которые можно почистить от знаков препинаний и создать кортеж вместо списков.

Текст должен быть обработан от символов, иначе получается слишком много слов за счет того что Hello! и Hello считаются разными словами.



In [299]:
def extract_words(data_column):
    sub_words = []
    for sublist in data_column:
        for word in sublist.split():
            sub_words.append(word.lower())

    text = ' '.join(sub_words)
    pattern = r'[^\w\s\'\.]'
    text_cleaned = re.sub(pattern, '', text)

    return text_cleaned

In [300]:
data['cleaned_up_subs'] = data['subs'].apply(extract_words)
data['cleaned_up_subs'][0]

"  walkietalkie squawks   rick morgan i don't know if you're out there. i don't know if you can hear me. maybe you're listening right now. i hope so. i found others my family if you can believe it.  static  my wife and son they're alive. i wanted you to know that. there's something else you need to know. atlanta isn't what we thought. it's not what they promised. the city is do not enter the city. it belongs to the dead now. we're camped a few miles northwest up by a big abandoned rock quarry. you can see it on a map. i hope you come find us. but be careful. last night walkers came out of the woods. we lost people. watch yourself morgan. take care of your boy. i'll try you again tomorrow at dawn.  static  andrea. i'm so sorry. she's gone. you got to let us take her. we all cared about her and i promise we'll be as gentle as we can.  grunts   coughing  she still won't move lori won't even talk to us. she's been there all night. what do we do can't just leave amy like that. we need to de

In [301]:
data['cleaned_up_subs'].apply(type)[0]

str

У нас получился новый признак data['used_words'] который содержит все слова, использованные в каждом фильме в формате str.

### Обработка библиотекой NLTK

In [302]:
# Функция для токенизации, лемматизации и удаления стоп-слов
def preprocess_text(text):
    # Токенизация
    tokens = word_tokenize(text)

    # Лемматизация слов
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]

    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in lemmatized_words
                      if word.lower()not in stop_words]

    return filtered_words

In [303]:
# Применение обработки к столбцу субтитров
data['preprocessed_subs'] = data['cleaned_up_subs'].apply(preprocess_text)
data['preprocessed_subs'].head()

0    [walkietalkie, squawk, rick, morgan, n't, know...
1    [people, yelling, radio, chatter, hey, hey, .,...
2    [bird, chirping, nothing, ., 's, nothing, ., '...
3    [thunder, rumbling, merle, 's, right, ., heard...
4    [bird, chirping, bug, chittering, boy, mom, ri...
Name: preprocessed_subs, dtype: object

Получили новый признак, содержащий обработанные субтитры, который можно будет использовать для получения векторов.

Вывод:
Мы создали датасет, содержащий следующие колонки:
 - subs, id, movie, level, _merge, unique_words.

Все данные относятся к типу object, числовые признаки отсутствуют.

В ходе соединения

Предобработка включала в себя:
- замена пропущенных значений заглушкой
Сделали предварительную очистку субтитров:
-


# Подготовка модели

### Баланс классов

In [304]:
data['level'].value_counts(normalize = True)

b2            0.416309
b1            0.227468
c1            0.167382
a2/a2+        0.107296
b1,_b2        0.034335
a2            0.025751
a2/a2+,_b1    0.021459
Name: level, dtype: float64

Для начала нам достаточно только три основных уровня А, B, C.


In [305]:
sorted(data['level'].unique())

['a2', 'a2/a2+', 'a2/a2+,_b1', 'b1', 'b1,_b2', 'b2', 'c1']

In [306]:
elementary = ['a2', 'a2/a2+', 'a2/a2+,_b1']
intermediate = ['b1', 'b1,_b2', 'b2']
advanced = ['c1']

In [307]:
'a3' in elementary

False

In [308]:
for level in data['level'].unique():
  if level in elementary:
    data.loc[data['level']==level, 'level'] = 'a'
  elif level in intermediate:
    data.loc[data['level']==level, 'level'] = 'b'
  else:
    data.loc[data['level']==level, 'level'] = 'c'

In [309]:
data['level'].value_counts(normalize = True).to_frame()

,level
b,0.678112
c,0.167382
a,0.154506


Распределение классов весьма логичное:
больше всего фильмов для среднего уровня языка, а для продвинутого или начинающего уровня фильмов не так много.

На основании этого мы уже можем выбирать метрику, для нас критичнее будет предсказание класса b. Следовательно наша метрика должна отражать количество верно предсказанных классов a,c, таким образом нужно использовать:
- recall или f1 меру
- ROC AUC score.

### Признаки

In [314]:
#Удалим лишние признаки
data.columns

Index(['subs', 'movie', 'level', 'cleaned_up_subs', 'preprocessed_subs'], dtype='object')

In [313]:
data = data.drop(['id', '_merge'], axis = 1)

Я создам 4 новых признака, которые основаны на кодировке субтитров разными библиотеками на столбцах 'cleaned_up_subs' и'preprocessed_subs':
- вектор созданный из словаря частотности слов встроенного Python модуля collections, bow_matrix (bag_of_words)
-вектор созданный трансформером TfidfTransformer, который позволяет определелить вес каждого слова.

#### BoW_matrix()

In [324]:
data['preprocessed_subs'].apply(type).head()

0    <class 'str'>
1    <class 'str'>
2    <class 'str'>
3    <class 'str'>
4    <class 'str'>
Name: preprocessed_subs, dtype: object

In [317]:
data['preprocessed_subs'] = data['preprocessed_subs'].apply(str)
data['preprocessed_subs'].apply(type).head()

0    <class 'str'>
1    <class 'str'>
2    <class 'str'>
3    <class 'str'>
4    <class 'str'>
Name: preprocessed_subs, dtype: object

In [319]:
values = []
all_words = set()

for text in data['preprocessed_subs']:
    all_words.update(text.split())

for text in data['preprocessed_subs']:
    word_counts = Counter(text.split())
    vector = [word_counts[word] for word in all_words]
    values.append(vector)
# Преобразуем список values в массив numpy
preprocessed_matrix = np.array(values)

print(preprocessed_matrix.shape)

(233, 49153)


In [321]:
values = []
all_words = set()

for text in data['cleaned_up_subs']:
    all_words.update(text.split())

for text in data['cleaned_up_subs']:
    word_counts = Counter(text.split())
    vector = [word_counts[word] for word in all_words]
    values.append(vector)
# Преобразуем список values в массив numpy
cleaned_up_matrix = np.array(values)

print(cleaned_up_matrix.shape)

(233, 73167)


#### CountVectorizer()

In [ ]:
count = CountVectorizer()
bag = count.fit_transform(np.array(list(data['preprocessed_subs'])))

In [ ]:
print (count.vocabulary_)

In [ ]:
print(type(bag))

In [ ]:
print(bag.toarray())

In [325]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tfidf_preprocessed_matrix = tfidf.fit_transform(bag)

In [326]:
tfidf_preprocessed_matrix.shape

(233, 43374)

In [339]:
data['tfidf_preprocessed_matrix'] = tfidf_preprocessed_matrix.toarray()

ValueError: ignored

In [328]:
# повторим шаги для признака cleaned_up
bag = count.fit_transform(np.array(list(data['cleaned_up_subs'])))
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tfidf_cleaned_up_matrix = tfidf.fit_transform(bag)
tfidf_cleaned_up_matrix.shape

(233, 46846)

In [335]:
data['tfidf_cleaned_up_matrix'] = tfidf_cleaned_up_matrix.toarray()

In [338]:
data.head()


,subs,movie,level,cleaned_up_subs,preprocessed_subs,tfidf_cleaned_up_matrix
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,a,walkietalkie squawks rick morgan i don't k...,"['walkietalkie', 'squawk', 'rick', 'morgan', ""...",0.0
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,a,people yelling radio chatter hey hey. wh...,"['people', 'yelling', 'radio', 'chatter', 'hey...",0.0
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,a,birds chirping what nothing. it's not noth...,"['bird', 'chirping', 'nothing', '.', ""'s"", 'no...",0.0
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,a,thunder rumbling merle that's right. you hea...,"['thunder', 'rumbling', 'merle', ""'s"", 'right'...",0.0
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,a,birds chirping bugs chittering boy mom r...,"['bird', 'chirping', 'bug', 'chittering', 'boy...",0.0
